# Importing libraries

In [115]:
import sys
from pathlib import Path
import glob
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse, r2_score as r2
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import RandomForestRegressor
import pickle
from utils import fragment_control, bounding_box, mkdir

# Reading descriptors

In [116]:
desc_dict = {}
for desc in glob.glob('../data/*.csv'):
    desc_dict[Path(desc).stem] = pd.read_csv(desc)

# Models for the computed rate of reaction (R<sub>QSPR</sub>)

## Reading hyperparameters of the best models

In [97]:
target_col = 'dG_MD_ylog'
descs_rf = pd.read_csv('../results/best_hyperparameters/dG_best_models_RF.csv')
descs_xgb = pd.read_csv('../results/best_hyperparameters/dG_best_models_XGB.csv')

In [98]:
model_res = desc_dict[descs_rf.loc[0, 'desc_name']].loc[:, ['cid', 'std_smiles', 'CompRate_ylog', 'dG_MD_ylog']]
for desc in descs_xgb['desc_name']:
    model_res[desc + '_XGB'] = None
for desc in descs_rf['desc_name']:
    model_res[desc + '_RF'] = None

## Evaluation of the models in 5 times repeated 5-fold CV

In [99]:
folds_stats = [] # list for saving the stats from CV folds

nspl = 5  # number of folds for CV procedure

nrpts = 5  # number of repeats of CV procedure

kf = RepeatedKFold(n_splits=nspl, n_repeats=nrpts, random_state=0)

i = 0

for train_index, test_index in kf.split(model_res):
   
    y_train = model_res.loc[train_index, f'{target_col}']
    df_temp = model_res.copy(deep=True)
    
    for desc_name in descs_xgb['desc_name']:
        
        X = desc_dict[desc_name].iloc[:, 4:]
        
        xgb_params = eval(descs_xgb.loc[descs_xgb['desc_name']==desc_name, f'{target_col}_param_XGB'].iloc[0])
        xgb_params.update({'n_jobs': -1})
        xgb_model = XGBRegressor(**xgb_params)
        xgb_model.fit(X.loc[train_index], y_train)
        
        df_temp.loc[test_index, f'{desc_name}_XGB'] = xgb_model.predict(X.loc[test_index])
        df_temp.loc[bounding_box(X.loc[train_index], X.loc[test_index]), f'{desc_name}_AD'] = 1   
        df_temp.loc[test_index, 'CV_fold'] = i
        
        q2_proper = r2(model_res.loc[test_index, f'{target_col}'], xgb_model.predict(X.loc[test_index]))
        
        print(f'Method: XGB; Q2: {q2_proper}')
        

    
    for desc_name in descs_rf['desc_name']:
        
        X = desc_dict[desc_name].iloc[:, 4:]
        
        rf_params = eval(descs_rf.loc[descs_rf['desc_name']==desc_name, f'{target_col}_param_RF'].iloc[0])
        rf_params.update({'n_jobs': -1})
        rf_model = RandomForestRegressor(**rf_params)
        rf_model.fit(X.loc[train_index], y_train)
        
        df_temp.loc[test_index, f'{desc_name}_RF'] = rf_model.predict(X.loc[test_index])
        
        q2_proper = r2(model_res.loc[test_index, f'{target_col}'], rf_model.predict(X.loc[test_index]))
        
        print(f'Method: RF; Q2: {q2_proper}')
        
        if desc_name not in descs_xgb['desc_name'].dropna().tolist():
            df_temp.loc[bounding_box(X.loc[train_index], X.loc[test_index]), f'{desc_name}_AD'] = 1
       
    folds_stats.append(df_temp.loc[~df_temp['CV_fold'].isna()])
    i+=1
    print(i)

Method: XGB; Q2: 0.789297748254274
Method: XGB; Q2: 0.8289143080020616
Method: XGB; Q2: 0.8020772225859079
Method: XGB; Q2: 0.7993462806091851
Method: XGB; Q2: 0.8489776090036177
Method: XGB; Q2: 0.80572816977805
Method: XGB; Q2: 0.8170655559683995
Method: XGB; Q2: 0.789297748254274
Method: XGB; Q2: 0.8020772225859079
Method: XGB; Q2: 0.7993462806091851
Method: XGB; Q2: 0.8489776090036177
Method: XGB; Q2: 0.8170655559683995
Method: RF; Q2: 0.8578690431529928
Method: RF; Q2: 0.8389239170531331
Method: RF; Q2: 0.8331969384626684
Method: RF; Q2: 0.8201655738654641
Method: RF; Q2: 0.8228944714692555
Method: RF; Q2: 0.826889172953377
Method: RF; Q2: 0.8245028041570099
Method: RF; Q2: 0.8349411047521782
Method: RF; Q2: 0.8459652025915254
Method: RF; Q2: 0.8384079181661315
Method: RF; Q2: 0.8414802893165443
Method: RF; Q2: 0.8334194774827002
Method: RF; Q2: 0.8223328096858665
Method: RF; Q2: 0.8314510918527791
Method: RF; Q2: 0.8269414874975942
Method: RF; Q2: 0.8333109083793088
1
Method: XGB

Method: XGB; Q2: 0.7946310810531062
Method: XGB; Q2: 0.736970095679558
Method: XGB; Q2: 0.7775286326993217
Method: XGB; Q2: 0.7648182514342684
Method: RF; Q2: 0.6965020922586844
Method: RF; Q2: 0.7532804105343025
Method: RF; Q2: 0.731417680753527
Method: RF; Q2: 0.7100925238125444
Method: RF; Q2: 0.7092864559409571
Method: RF; Q2: 0.7179493767828784
Method: RF; Q2: 0.7309229046738729
Method: RF; Q2: 0.752330256927284
Method: RF; Q2: 0.7309060184559575
Method: RF; Q2: 0.7297058892293712
Method: RF; Q2: 0.7377819694296115
Method: RF; Q2: 0.7345554014209221
Method: RF; Q2: 0.7165124295096784
Method: RF; Q2: 0.7255038671661
Method: RF; Q2: 0.7276763556583412
Method: RF; Q2: 0.7214216830796241
9
Method: XGB; Q2: 0.7378061450736032
Method: XGB; Q2: 0.6945364951757329
Method: XGB; Q2: 0.7507709943150563
Method: XGB; Q2: 0.7479150419713518
Method: XGB; Q2: 0.7483608088250978
Method: XGB; Q2: 0.7041379090516962
Method: XGB; Q2: 0.7590125650144928
Method: XGB; Q2: 0.7378061450736032
Method: XGB;

Method: RF; Q2: 0.7668866294733517
Method: RF; Q2: 0.7867443132456663
Method: RF; Q2: 0.7829578294326828
Method: RF; Q2: 0.7930792981325959
Method: RF; Q2: 0.7668255284483343
Method: RF; Q2: 0.7965689585781544
Method: RF; Q2: 0.7870391299406464
Method: RF; Q2: 0.7698225136408071
Method: RF; Q2: 0.774870126669718
Method: RF; Q2: 0.7697243702629184
Method: RF; Q2: 0.7930303008859463
Method: RF; Q2: 0.7810419714106218
17
Method: XGB; Q2: 0.8221930236391768
Method: XGB; Q2: 0.8324224181408659
Method: XGB; Q2: 0.8116870626300006
Method: XGB; Q2: 0.8133018980168459
Method: XGB; Q2: 0.8491274699545593
Method: XGB; Q2: 0.8363692112759874
Method: XGB; Q2: 0.8436464648910451
Method: XGB; Q2: 0.8221930236391768
Method: XGB; Q2: 0.8116870626300006
Method: XGB; Q2: 0.8133018980168459
Method: XGB; Q2: 0.8491274699545593
Method: XGB; Q2: 0.8436464648910451
Method: RF; Q2: 0.823429227489995
Method: RF; Q2: 0.813458712778032
Method: RF; Q2: 0.8067856984528183
Method: RF; Q2: 0.7171620648500049
Method: 

Method: RF; Q2: 0.843388829475799
Method: RF; Q2: 0.8555211159670282
Method: RF; Q2: 0.8559526175059581
Method: RF; Q2: 0.8429811893156672
25


In [100]:
all_stats = pd.concat(folds_stats, axis=0)  # concatenating the stat data from folds
pred_columns = [col for col in all_stats.columns if '_RF' in col or '_XGB' in col]  # list of columns with predicted values

### Consideration of the applicability domain (AD)

In [101]:
ad_columns = [col for col in all_stats.columns if 'AD' in col]  # list of columns with applicability domain info
all_stats.loc[:, ad_columns] = all_stats.loc[:, ad_columns].fillna(0)  # replace None values with 0
all_stats['applicability_domain'] = None  # column to check whether the compound is in AD
all_stats['applicability_domain_conf'] = None  # for how many models compound is in AD
all_stats['applicability_domain_conf'] = (all_stats.loc[:, ad_columns]==0).astype(int).sum(axis=1)  # sum number of descriptor sets for which compound is in AD
all_stats.loc[all_stats['applicability_domain_conf']>=3, 'applicability_domain'] = 1  # if compound is inside AD for at least 3 descriptor sets => 1
all_stats['applicability_domain'] = all_stats['applicability_domain'].fillna(0)

In [102]:
all_stats

,cid,std_smiles,CompRate_ylog,dG_MD_ylog,IIRA14+OPERA_XGB,IIRA14P+OPERA_XGB,IIRA15+OPERA_XGB,IIRA16+OPERA_XGB,IIRA24+OPERA_XGB,IIRA24P+OPERA_XGB,...,IIRAB24+OPERA_AD,IIRAB25+OPERA_AD,IIRA13+OPERA_AD,IIRA17+OPERA_AD,IIRA23+OPERA_AD,IIRAB13+OPERA_AD,IIRAB17+OPERA_AD,IIRAB23+OPERA_AD,applicability_domain,applicability_domain_conf
2,100-37-8,CCN(CC)CCO,-0.684356,-1.806987,-1.793912,-1.786249,-1.797403,-1.785379,-1.794173,-1.787515,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,18
7,7005-47-2,CN(C)C(C)(C)CO,-0.396975,-1.779119,-1.782521,-1.785056,-1.788471,-1.780311,-1.780552,-1.781792,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0
8,19059-68-8,CN(C)CC(C)(C)CO,-0.356186,-1.761684,-1.784903,-1.785115,-1.788102,-1.775545,-1.784456,-1.775175,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,18
10,139-87-7,CCN(CCO)CCO,-0.176428,-1.720622,-1.745253,-1.730112,-1.732022,-1.733621,-1.725113,-1.748068,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,18
16,2955-88-6,OCCN1CCCC1,-0.679613,-1.801336,-1.789796,-1.788106,-1.790278,-1.788635,-1.795122,-1.789138,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,59941-12-7,CCCCCC1CCCN1CCO,-0.382947,-1.760751,-1.745565,-1.749897,-1.743734,-1.750871,-1.744259,-1.742109,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1,10
117,64897-89-8,CC(C)CCN(CCO)CCO,-0.366887,-1.760357,-1.762424,-1.761087,-1.766255,-1.762741,-1.756962,-1.766497,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,18
118,40694-17-5,CN(CCO)CCCCCO,-0.379963,-1.769528,-1.795883,-1.801389,-1.801322,-1.805404,-1.796566,-1.80598,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1,10
119,102-81-8,CCCCN(CCO)CCCC,-0.549957,-1.799773,-1.790483,-1.793426,-1.79772,-1.799242,-1.794147,-1.799906,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,0


In [103]:
all_stats.groupby('cid')['applicability_domain'].mean().value_counts()  # how many compounds are outside AD in CV procedure?

1.0    75
0.0    35
0.8     8
0.6     5
0.4     1
Name: applicability_domain, dtype: int64

In [104]:
all_stats_temp = all_stats.copy(deep=True)  # create a temporary dataframe for aggregation of predictions cosnidering AD
for col in ad_columns:
    all_stats_temp.loc[all_stats_temp[col]==1, [coli for coli in all_stats_temp.columns if col.replace('_AD', '_RF') in coli or col.replace('_AD', '_XGB') in coli]] = None
all_stats_temp['avg_pred'] = all_stats_temp.loc[:, pred_columns].median(axis=1)
all_stats.loc[:, 'avg_pred'] = all_stats_temp['avg_pred']
all_stats.loc[:, 'avg_pred_AD'] = all_stats_temp['avg_pred']
all_stats.loc[all_stats['avg_pred_AD'].isna(), 'avg_pred'] = all_stats.loc[all_stats['avg_pred_AD'].isna(), pred_columns].median(axis=1)

In [105]:
all_stats

,cid,std_smiles,CompRate_ylog,dG_MD_ylog,IIRA14+OPERA_XGB,IIRA14P+OPERA_XGB,IIRA15+OPERA_XGB,IIRA16+OPERA_XGB,IIRA24+OPERA_XGB,IIRA24P+OPERA_XGB,...,IIRA13+OPERA_AD,IIRA17+OPERA_AD,IIRA23+OPERA_AD,IIRAB13+OPERA_AD,IIRAB17+OPERA_AD,IIRAB23+OPERA_AD,applicability_domain,applicability_domain_conf,avg_pred,avg_pred_AD
2,100-37-8,CCN(CC)CCO,-0.684356,-1.806987,-1.793912,-1.786249,-1.797403,-1.785379,-1.794173,-1.787515,...,0.0,0.0,0.0,0.0,0.0,0.0,1,18,-1.790484,-1.790484
7,7005-47-2,CN(C)C(C)(C)CO,-0.396975,-1.779119,-1.782521,-1.785056,-1.788471,-1.780311,-1.780552,-1.781792,...,1.0,1.0,1.0,1.0,1.0,1.0,0,0,-1.785142,NaN
8,19059-68-8,CN(C)CC(C)(C)CO,-0.356186,-1.761684,-1.784903,-1.785115,-1.788102,-1.775545,-1.784456,-1.775175,...,0.0,0.0,0.0,0.0,0.0,0.0,1,18,-1.782332,-1.782332
10,139-87-7,CCN(CCO)CCO,-0.176428,-1.720622,-1.745253,-1.730112,-1.732022,-1.733621,-1.725113,-1.748068,...,0.0,0.0,0.0,0.0,0.0,0.0,1,18,-1.730146,-1.730146
16,2955-88-6,OCCN1CCCC1,-0.679613,-1.801336,-1.789796,-1.788106,-1.790278,-1.788635,-1.795122,-1.789138,...,0.0,0.0,0.0,0.0,0.0,0.0,1,18,-1.790268,-1.790268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,59941-12-7,CCCCCC1CCCN1CCO,-0.382947,-1.760751,-1.745565,-1.749897,-1.743734,-1.750871,-1.744259,-1.742109,...,0.0,1.0,0.0,0.0,1.0,0.0,1,10,-1.750029,-1.750029
117,64897-89-8,CC(C)CCN(CCO)CCO,-0.366887,-1.760357,-1.762424,-1.761087,-1.766255,-1.762741,-1.756962,-1.766497,...,0.0,0.0,0.0,0.0,0.0,0.0,1,18,-1.759635,-1.759635
118,40694-17-5,CN(CCO)CCCCCO,-0.379963,-1.769528,-1.795883,-1.801389,-1.801322,-1.805404,-1.796566,-1.80598,...,0.0,1.0,0.0,0.0,1.0,0.0,1,10,-1.798984,-1.798984
119,102-81-8,CCCCN(CCO)CCCC,-0.549957,-1.799773,-1.790483,-1.793426,-1.79772,-1.799242,-1.794147,-1.799906,...,1.0,1.0,1.0,1.0,1.0,1.0,0,0,-1.797029,NaN


### Showing stats obtained in 5x5 CV

In [106]:
q2_cv = []
mae_cv = []
rmse_cv = []
all_stats_res = []
for cv_fold in range(0, nspl*nrpts, nspl):
    if cv_fold == 0:
        all_stats_res.append(all_stats.loc[all_stats['CV_fold'].isin([i for i in range(cv_fold, cv_fold+nspl)]), ['cid', 'avg_pred']])
    else:
        all_stats_res.append(all_stats.loc[all_stats['CV_fold'].isin([i for i in range(cv_fold, cv_fold+nspl)]), ['avg_pred']])
    q2_cv.append(r2(all_stats.loc[all_stats['CV_fold'].isin([i for i in range(cv_fold, cv_fold+nspl)]), [f'{target_col}']], all_stats.loc[all_stats['CV_fold'].isin([i for i in range(cv_fold, cv_fold+nspl)]), ['avg_pred']]))
    rmse_cv.append(mse(all_stats.loc[all_stats['CV_fold'].isin([i for i in range(cv_fold, cv_fold+nspl)]), [f'{target_col}']], all_stats.loc[all_stats['CV_fold'].isin([i for i in range(cv_fold, cv_fold+nspl)]), ['avg_pred']])**0.5)             
    mae_cv.append(mae(all_stats.loc[all_stats['CV_fold'].isin([i for i in range(cv_fold, cv_fold+nspl)]), [f'{target_col}']], all_stats.loc[all_stats['CV_fold'].isin([i for i in range(cv_fold, cv_fold+nspl)]), ['avg_pred']]))                         

In [107]:
print(f'Gas: {target_col}; Q2(5x5CV): {round(np.mean(q2_cv), 2)}; Q2_std(5x5CV): {round(np.std(q2_cv), 2)}')
print(f'Gas: {target_col}; RMSE(5x5CV): {round(np.mean(rmse_cv), 2)}; RMSE_std(5x5CV): {round(np.std(rmse_cv), 3)}')
print(f'Gas: {target_col}; MAE(5x5CV): {round(np.mean(mae_cv), 2)}; MAE_std(5x5CV): {round(np.std(mae_cv), 3)}')

Gas: dG_MD_ylog; Q2(5x5CV): 0.77; Q2_std(5x5CV): 0.01
Gas: dG_MD_ylog; RMSE(5x5CV): 0.02; RMSE_std(5x5CV): 0.001
Gas: dG_MD_ylog; MAE(5x5CV): 0.02; MAE_std(5x5CV): 0.0


### Analyzing outliers

In [108]:
rmse_cv = np.mean(rmse_cv)  # the value of rmse from CV for outlier analysis

In [109]:
rmse_cv*2

0.04652467567380023

In [110]:
all_stats_res = pd.concat(all_stats_res, axis=1)
all_stats_res = pd.merge(all_stats_res, model_res.loc[:, ['cid', f'{target_col}']], on='cid', how='left')
all_stats_res.loc[:, 'avg_pred_fin'] = all_stats_res.loc[:, ['avg_pred']].mean(axis=1)
all_stats_res['abs_err'] = all_stats_res.loc[:, 'avg_pred_fin'] - all_stats_res.loc[:, f'{target_col}']
all_stats_res['abs_err'] = all_stats_res['abs_err'].abs()

In [111]:
all_stats_res.loc[all_stats_res['abs_err']>=2*rmse_cv, 'outlier'] = 1

In [112]:
all_stats_res.sort_values('abs_err', ascending=False).head(10)

,cid,avg_pred,avg_pred,avg_pred,avg_pred,avg_pred,dG_MD_ylog,avg_pred_fin,abs_err,outlier
78,854665-75-1,-1.668240,-1.662061,-1.661589,-1.666072,-1.658940,-1.584181,-1.663380,0.079199,1.0
22,13444-24-1,-1.801313,-1.801523,-1.802150,-1.801279,-1.796356,-1.739990,-1.800524,0.060534,1.0
14,621-56-7,-1.772839,-1.773786,-1.774942,-1.773066,-1.776268,-1.828319,-1.774180,0.054139,1.0
84,15520-05-5,-1.719747,-1.719073,-1.718045,-1.712786,-1.711269,-1.662391,-1.716184,0.053793,1.0
39,86375-49-7,-1.747357,-1.753544,-1.754867,-1.747710,-1.747933,-1.800448,-1.750282,0.050166,1.0
66,13297-91-1,-1.696025,-1.694134,-1.692491,-1.691624,-1.684646,-1.642078,-1.691784,0.049706,1.0
103,20966-69-2,-1.737994,-1.737264,-1.740728,-1.740301,-1.745120,-1.785487,-1.740282,0.045206,NaN
121,5842-08-0,-1.804658,-1.801221,-1.803420,-1.797312,-1.807982,-1.847296,-1.802919,0.044377,NaN
47,6039-36-7,-1.780739,-1.779951,-1.774643,-1.775449,-1.788526,-1.823743,-1.779862,0.043882,NaN
23,105-59-9,-1.715282,-1.713183,-1.711704,-1.714304,-1.718664,-1.758130,-1.714628,0.043503,NaN


### Saving the dataframe with stats for the figure

In [ ]:
all_stats_res.to_csv('../results/model_stats/dG_5x5CV_stats.csv', index=False)

## (re)Fitting and saving the models

In [114]:
y = model_res.loc[:, f'{target_col}']
for desc_name in descs_xgb['desc_name']:
    X = desc_dict[desc_name].iloc[:, 4:]
    xgb_params = eval(descs_xgb.loc[descs_xgb['desc_name']==desc_name, f'{target_col}_param_XGB'].iloc[0])
    xgb_params.update({'n_jobs': -1})
    xgb_model = XGBRegressor(**xgb_params)
    xgb_model.fit(X, y)
    with open(f'../results/models/dg/dG_{desc_name}_xgb.pkl', 'wb') as mf:
        pickle.dump(xgb_model, mf)
for desc_name in descs_rf['desc_name']:
    X = desc_dict[desc_name].iloc[:, 4:]
    rf_params = eval(descs_rf.loc[descs_rf['desc_name']==desc_name, f'{target_col}_param_RF'].iloc[0])
    rf_params.update({'n_jobs': -1})
    rf_model = RandomForestRegressor(**rf_params)
    rf_model.fit(X, y)
    with open(f'../results/models/dg/dG_{desc_name}_rf.pkl', 'wb') as mf:
        pickle.dump(rf_model, mf)